###

Connecting to drive

###

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

###
###   DOWLOAD IMAGE OR ANY FILE
###

In [0]:
from PIL import Image

import matplotlib
file_name = 'out2.png'
matplotlib.image.imsave(file_name, img)

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(file_name)

###

Tensorflow image loading,resizing, preprocessing functions

###

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
keras = tf.keras

#Takes path to file as input
def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  w = tf.shape(image)[1]

  w = w // 2
  input_image = image[:, w:, :]

  input_image = tf.cast(input_image, tf.float32)  
  return input_image


def resize(input_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image

# normalizing the images to [-1, 1]

def normalize(input_image):
    input_image = (input_image / 127.5) - 1
    return input_image

def preprocess_imgs(input_image):
    size=256
    input_image=resize(input,size,size)
    input_image = np.expand_dims(input_image, axis=0)
    
    ###
    #Not using normalize function as it is done internanlly the step below
    ###
    
    input_image = keras.applications.ResNet50.preprocess_inputpreprocess_input(x)

    return input_image
  
"""

####
Various models present in tf2.0
####

DenseNet121(...): Instantiates the DenseNet architecture.

DenseNet169(...): Instantiates the DenseNet architecture.

DenseNet201(...): Instantiates the DenseNet architecture.

InceptionResNetV2(...): Instantiates the Inception-ResNet v2 architecture.

InceptionV3(...): Instantiates the Inception v3 architecture.

MobileNet(...): Instantiates the MobileNet architecture.

NASNetLarge(...): Instantiates a NASNet model in ImageNet mode.

NASNetMobile(...): Instantiates a Mobile NASNet model in ImageNet mode.

ResNet50(...): Instantiates the ResNet50 architecture.

VGG16(...): Instantiates the VGG16 architecture.

VGG19(...): Instantiates the VGG19 architecture.

Xception(...): Instantiates the Xception architecture.

"""

In [0]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,    
    RandomGamma,
    RGBShift
)

In [0]:
original_height=224
original_width=224
aug = Compose([
    OneOf([RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
          PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)], p=1),    
    VerticalFlip(p=0.5),              
    RandomRotate90(p=0.5),
    OneOf([
        ElasticTransform(p=0.4, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        GridDistortion(p=0.3),
        OpticalDistortion(p=0.4, distort_limit=2, shift_limit=0.5)                  
        ], p=0.8),
    CLAHE(p=0.8),
    RandomBrightnessContrast(p=0.8),
    RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, always_apply=False, p=0.9),
    RandomGamma(p=0.8)])



In [0]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

In [0]:
augmented = aug(image=image, mask=mask)

image_elastic = augmented['image']
mask_elastic = augmented['mask']

visualize(image_elastic, mask_elastic, original_image=image, original_mask=mask)

Batch Generator, using above preprocessing functions.

In [0]:
def image_generator(input_ids, batch_size = 32,is_training=0):
  
  while True:
    batch_paths = np.random.choice(a= input_ids, size = batch_size)
    
    batch_input = []
    batch_output = []
    
    for input_id in batch_paths:
      output = cv2.imread(join(path_gt, input_id+'-gt.pbm'))
      input = cv2.imread(join(path_org, input_id+'-org.jpg'))
      
      input = preprocess_image(input) 
      output = preprocess_image(output) 
      if is_training:
        augmented = aug(image=input, mask=output)
        input=augmented['image']
        output=augmented['mask']

      
      batch_input += [input]
      batch_output += [output]
   
    batch_x = np.array(batch_input)
    batch_y = np.array(batch_output)
    
    
    
    yield (batch_x, batch_y)
      